In [18]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
pd.options.display.max_columns = 200
#nltk.download()

def setNumberRangeCat(inputString):
    tempStr = str(inputString)
    #print(" number range cat var=" + str(tempStr) + ';index=' + str(indexT))
    if tempStr == 'DESPAIR':
        out = 0
    else:
        if tempStr == 'MEH':
            out = 1
        else:
            if tempStr == 'JOY':
                out = 2
            else:
                out =1 # this is 'unk'
    return out

def cleanStr(row):
    return re.sub('[^A-Za-z0-9]+', ' ', str(row).lower()) # remove special chars

def cleanStrSp(row):
    out =[]
    for inst in row:
        tmp = re.sub('[^A-Za-z0-9]+', ' ', str(inst)) # remove special chars
        out.append(re.sub(r"^\s+|\s+" , "" , tmp)) # remove leading and trailing spaces
    return out

def dropSpacesTruncate(row):
    return row.replace(" ", "")[:10] #clean out internal spaces

def isInt_float(s):
    try:
        return float(str(s)).is_integer()
    except:
        return False

def fixAge(row):
    if str(row[3]) == 'unk':
        out = 40 # set to median val
    else:
        if isInt_float(row[3]):
            out = int(row[3])
        else:
            out = 40 # set to median value
    return out
        
def setBinary(row):
    if row == 'unk':
        out = 0
    else:
        if row =='1':
            out = 1
        else:
            out = 0
    return out

def setDressBinary(row):
    if row == 'Blue and black':
        out = 0
    else:
        if row =='White and gold':
            out = 1
        else:
            out = 1 # most common answer will be the default
    return out

usaList =['USA','usa','Us','US','us','America','america','UnitedStates','unitedstates','Unitedstates','TheUnitedStates','USofA']
def createOneHotUSA(eventDesc):
    #print('EventDesc' + eventDesc)
    res = any(ele in eventDesc[4] for ele in usaList)
    return (1 if res == True else 0)

        
canList =['Canada','canada','can']        
def createOneHotCanada(eventDesc):
    #print('EventDesc' + eventDesc)
    res = any(ele in eventDesc[4] for ele in canList)
    return (1 if res == True else 0)

ukList =['UK','uk','UnitedKingdom','England','Ireland','Scotland']
def createOneHotUK(eventDesc):
    #print('EventDesc' + eventDesc)
    res = any(ele in eventDesc[4] for ele in ukList)
    return (1 if res == True else 0)
        
excelTmp = pd.ExcelFile("candyhierarchy2017.xlsx") 
candyDF = pd.read_excel(excelTmp, dtype = str)  #did dtype str because many columns are mixed types
candyDF.fillna('unk', inplace=True)  #choice of no answer is still information

print("read into memory data set candyhierarchy2017.xlsx")
print("candyDF shape =", candyDF.shape)
candyDF.dropna()  #just checking... shouldn't be any
print('After dropping NA missing values:')
print("candyDF shape =", candyDF.shape)
#print first 10 rows
print(candyDF[:10])


read into memory data set candyhierarchy2017.xlsx
candyDF shape = (2460, 120)
After dropping NA missing values:
candyDF shape = (2460, 120)
  Internal ID Q1: GOING OUT? Q2: GENDER Q3: AGE Q4: COUNTRY  \
0    90258773            unk        unk     unk         unk   
1    90272821             No       Male      44        USA    
2    90272829            unk       Male      49         USA   
3    90272840             No       Male      40          us   
4    90272841             No       Male      23         usa   
5    90272852             No       Male     unk         unk   
6    90272853             No       Male      53         usa   
7    90272854             No       Male      33      canada   
8    90272858             No       Male      40      Canada   
9    90272859             No     Female      53          Us   

  Q5: STATE, PROVINCE, COUNTY, ETC Q6 | 100 Grand Bar  \
0                              unk                unk   
1                               NM                ME

In [26]:
stopL=set(stopwords.words('english'))
colList = candyDF.columns
newCols = []
for nameT in colList:
    nameT = cleanStr(nameT)
    nameT = nameT.split()
    nameT = ''.join((filter(lambda s: s not in stopL, nameT)))
    newCols.append(dropSpacesTruncate(nameT))
candyDF.columns = newCols
candyDF.describe()

,q6100grand,q6anonymou,q6fullsize,q6blackjac,q6bonkersc,q6bonkersb,q6bottleca,q6boxraisi,q6brokengl,q6butterfi,q6cadburyc,q6candycor,q6candycle,q6caramell,q6cashform,q6chardonn,q6chicksti,q6chiclets,q6coffeecr,q6creepyre,q6dentalpa,q6dots,q6dovebars,q6fuzzypea,q6genericb,q6glowstic,q6googoocl,q6goodnple,q6gumbaseb,q6gummybea,q6hardcand,q6healthyf,q6heathbar,q6hersheyd,q6hersheym,q6hersheyk,q6hugsactu
count,2460.000000,2460.000000,2460.000000,2460.000000,2460.000000,2460.000000,2460.000000,2460.000000,2460.000000,2460.000000,2460.000000,2460.000000,2460.000000,2460.000000,2460.000000,2460.000000,2460.000000,2460.000000,2460.000000,2460.000000,2460.000000,2460.000000,2460.000000,2460.000000,2460.000000,2460.000000,2460.000000,2460.000000,2460.000000,2460.000000,2460.000000,2460.000000,2460.000000,2460.000000,2460.000000,2460.000000,2460.000000
mean,1.320325,0.628862,1.626829,0.715041,0.845935,0.856504,0.961382,0.568293,0.349187,1.421138,1.172764,0.892276,0.477236,1.321138,1.558537,1.293902,0.859756,0.789837,0.972764,0.528455,0.454472,0.896748,1.434146,0.888618,0.587805,1.066667,0.977642,0.864228,0.423577,1.128862,0.872358,0.745122,1.347967,1.378049,1.272764,1.248780,0.814634
std,0.535667,0.613708,0.497832,0.527890,0.474056,0.529083,0.644472,0.583606,0.496858,0.598799,0.681354,0.696372,0.530380,0.592175,0.547303,0.682229,0.603169,0.603752,0.641894,0.670353,0.567487,0.652762,0.577584,0.652670,0.611678,0.653164,0.595418,0.670245,0.529193,0.680332,0.576735,0.612726,0.632629,0.614457,0.617613,0.606239,0.709396
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
50%,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
75%,2.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000,1.000000,1.000000,2.000000,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,2.000000,2.000000,2.000000,2.000000,1.000000
max,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000


In [27]:
# drop complicated or extraneous columns
colList = candyDF.columns
print(colList)
candyDF = candyDF.drop(['q7joy', 'q8despair', 'q9comments', 'unnamed113', 'clickcoord'], axis=1)
print(candyDF.shape)
candyDF.describe()

Index(['internalid', 'q1going', 'q2gender', 'q3age', 'q4country', 'q5statepro',
       'q6100grand', 'q6anonymou', 'q6fullsize', 'q6blackjac',
       ...
       'q6whatcham', 'q6whitebre', 'q6wholewhe', 'q6yorkpepp', 'q10dress',
       'q11day', 'q12mediada', 'q12mediasc', 'q12mediaes', 'q12mediaya'],
      dtype='object', length=115)


KeyError: "['q7joy' 'q8despair' 'q9comments' 'unnamed113' 'clickcoord'] not found in axis"

In [28]:
pd.DataFrame((candyDF[0:119] == 'unk').sum()).to_csv('sumCandyDFColumns.csv')

/home/greg/anaconda3/envs/BigDataPy36_env/lib/python3.6/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [29]:
candyDF['q1going'] = cleanStrSp(candyDF['q1going'])
candyDF['q4country'] = cleanStrSp(candyDF['q4country'])
candyDF['q5statepro'] = cleanStrSp(candyDF['q5statepro'])

In [30]:
candyDF.describe()
candyDF.shape
#print(candyDF[:10])
print(candyDF.q4country.unique())
print(candyDF.q5statepro.unique())

['unk' 'USA' 'us' 'usa' 'canada' 'Canada' 'Us' 'US' 'Murica'
 'UnitedStates' 'uk' 'UnitedKingdom' 'unitedstates' 'Usa' 'Unitedstaes'
 'UnitedStatesofAmerica' 'UAE' 'England' 'UK' 'Mexico' 'Unitedstates'
 'USAUSAUSA' 'america' '35' 'france' 'finland' 'unhingedstates'
 'unitedstatesofamerica' 'USofA' 'UnitesStates' 'TheUnitedStates'
 'NorthCarolina' 'UniedStates' 'Netherlands' 'germany' 'Europe' 'Earth'
 'CostaRica' 'TheUnitedStatesofAmerica' 'unitestates' '46' 'cascadia'
 'Australia' 'insanitylately' 'Greece' 'USAHardtotellanymore' 'merica'
 'usas' 'Pittsburgh' '45' 'UnitedState' '32' 'France' 'australia' 'A'
 'Can' 'Canae' 'NewYork' 'Trumpistan' 'Ireland' 'UnitedSates' 'Korea'
 'California' 'Japan' 'USa' 'Southafrica'
 'IpretendtobefromCanadabutIamreallyfromtheUnitedStates' 'Uk' 'Iceland'
 'Germany' 'Scotland' 'Denmark' 'UnitedStated' 'Switzerland' 'AhemAmerca'
 'UD' 'SouthKorea' 'NewJersey' 'CANADA' 'Indonesia' 'Unitedststes'
 'America' 'TheNetherlands' 'UnitedStatss' 'endland' 'Atlan

In [31]:
mehColList = ['q6100grand', 'q6anonymou','q6fullsize','q6blackjac','q6bonkersc','q6bonkersb','q6bottleca','q6boxraisi','q6brokengl','q6butterfi','q6cadburyc','q6candycor','q6candycle','q6caramell','q6cashform','q6chardonn','q6chicksti','q6chiclets','q6coffeecr','q6creepyre','q6dentalpa','q6dots','q6dovebars','q6fuzzypea','q6genericb','q6glowstic','q6googoocl','q6goodnple','q6gumbaseb','q6gummybea','q6hardcand','q6healthyf','q6heathbar','q6hersheyd','q6hersheym','q6hersheyk','q6hugsactu','q6jollyran','q6jollyran','q6joyjoymi','q6juniormi','q6seniormi','q6kalesmoo','q6kinderha','q6kitkat','q6laffytaf','q6lemonhea','q6licorice','q6licorice','q6lindttru','q6lollipop','q6mars','q6maynards','q6mikeike','q6milkduds','q6milkyway','q6regularm','q6peanut','q6blue','q6red','q6greenpar','q6independ','q6abstaine','q6minibags','q6mintkiss','q6mintjule','q6mrgoodba','q6neccowaf','q6nerds','q6nestlecr','q6nlaters','q6peeps','q6pencils','q6pixystix','q6realhous','q6reesepea','q6reesepie','q6reggieja','q6rolos','q6sandwich','q6skittles','q6smarties','q6smarties','q6snickers','q6sourpatc','q6spottedd','q6starburs','q6sweettar','q6swedishf','q6sweetums','q6take5','q6tictacs','q6oddmarsh','q6threemus','q6tolberon','q6trailmix','q6twix','q6vialspur','q6vicodin','q6whatcham','q6whitebre','q6wholewhe','q6yorkpepp']

for colT in mehColList:
    candyDF[colT] = candyDF[colT].apply((lambda x: setNumberRangeCat(x[mehColList.index(colT)+6])), axis=1)

candyDF['usaOneH'] = candyDF.apply(createOneHotUSA, axis=1)
candyDF['canOneH'] = candyDF.apply(createOneHotCanada, axis=1)
candyDF['ukOneH'] = candyDF.apply(createOneHotUK, axis=1)
candyDF = candyDF.drop(['q4country','q5statepro'], axis=1)  # state province etc... has a huge variation across it based on unique values, dropping it
candyDF['q3age'] = candyDF.apply(fixAge, axis=1)
print(candyDF[:10])
result = candyDF.to_csv('MeyerCleanCandyDF.csv')

TypeError: <lambda>() got an unexpected keyword argument 'axis'